<a href="https://colab.research.google.com/github/srikar999/TakeoutDataAnalytics/blob/main/Gpay_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install pyspark

In [95]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [96]:
spark = SparkSession.builder.master("local").appName("Gpay_nb").getOrCreate()

In [97]:
import os
from pyspark.sql.types import DoubleType,IntegerType

In [98]:
inp="/content/drive/MyDrive/Takeout_Project/input/Takeout/Google Pay/Google transactions/"
oup="/content/drive/MyDrive/Takeout_Project/outputs/Gpay/"
txns=spark.read.option("inferSchema",True).option("charset", "ISO-8859-1").csv(inp,header=True)
txns=txns.withColumn("year",F.trim(F.split(F.col("Time"),",")[1]))
txns=txns.withColumn("amt",F.regexp_replace(F.col("Amount"), "[^A-Z0-9_.]", ""))
txns=txns.withColumn("amt_d",F.trim(F.split(F.col("amt"),"INR")[1]).cast(DoubleType()))
txns_agg=txns.groupBy("Product","Payment method","Status","year").agg(F.sum("amt_d").alias("amount"))
txns_agg_p=txns_agg.toPandas()
txns_agg_p.to_csv(oup+"google_txns_data.csv",index=False)

In [99]:
rewards_ip="/content/drive/MyDrive/Takeout_Project/input/Takeout/Google Pay/Rewards earned/Cashback Rewards.csv"
rewards=spark.read.option("inferSchema",True).option("charset", "ISO-8859-1").csv(rewards_ip,header=True)
rewards=rewards.withColumn("Date",F.to_timestamp(F.col("Date")))
rewards=rewards.groupBy("Date").agg(F.sum("Reward Amount").alias("amount"))
rewards=rewards.withColumn("year",F.year(F.col("Date"))).withColumn("month",F.month(F.col("Date")))
rewards_p=rewards.toPandas()
rewards_p.to_csv(oup+"rewards_data.csv",index=False)